In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
%matplotlib inline

In [34]:
new_donations=pd.read_pickle('new_donations.pkl')

In [35]:
new_donations.head()

,cohort_gift_pattern,amount_yr0,amount_yr1,amount_yr2,amount_yr3,amount_yr4,amount_2013,amount_2014,amount_2015,amount_2016,...,recovered_multi_year,inconsistent_multi_year,recovered_key_multi_year,lapsed_after_first,lapsed_after_first_long,lapsed_after_second,lapsed_multi_year,lapsed_long_multi_year,lapsed_key_multi_year,lapsed_long
0,"0,1,2,3,",1821138,4008515,6264498,1882613,0,0,0,1821138,4008515,...,0,0,0,0,0,0,0,0,0,0
1,"0,",2500,0,0,0,0,0,0,0,2500,...,0,0,0,0,1,0,0,0,0,0
2,"0,1,",2000,4000,0,0,0,0,0,2000,4000,...,0,0,0,0,0,0,0,0,0,1
3,"0,",2500,0,0,0,0,0,0,0,2500,...,0,0,0,0,1,0,0,0,0,0
4,"0,1,3,",7000,2500,0,10000,0,0,0,7000,2500,...,0,1,0,0,0,0,0,0,0,0


In [36]:
new_donations.first_gift_year.value_counts()

2016-01-01    445598
2017-01-01    434991
2015-01-01    355137
2014-01-01    320673
2013-01-01    320069
2018-01-01    147945
2012-01-01       141
Name: first_gift_year, dtype: int64

In [15]:
new_donations.cohort_gift_pattern.value_counts()

0,            1696680
0,1,           162004
0,2,            44403
0,1,2,          37639
0,3,            17619
0,1,2,3,        13781
0,1,2,3,4,      13102
0,1,3,           8134
0,2,3,           7626
0,4,             6912
0,3,4            3108
0,1,4,           2994
0,1,2,4,         2967
0,1,3,4,         2815
0,2,3,4,         2735
0,2,4,           2035
Name: cohort_gift_pattern, dtype: int64

In [22]:
print(f'# who gave year0: ',new_donations.query('amount_yr0 > 0').count()['id'])
print(f'# who gave year1: ',new_donations.query('amount_yr1 > 0').count()['id'])
print(f'# who gave year2: ',new_donations.query('amount_yr2 > 0').count()['id'])
print(f'# who gave year2 again: ', new_donations.query('amount_yr0 >  0 & amount_yr1 > 0 & amount_yr2 > 0').count()['id'])
print(f'# who gave year2 recover: ', new_donations.query('amount_yr0 >  0 & amount_yr1 == 0 & amount_yr2 > 0').count()['id'])
print(f'# who gave year3: ',new_donations.query('amount_yr3 > 0').count()['id'])
print(f'# who gave year3 again: ', new_donations.query('amount_yr0 >  0 & amount_yr1 > 0 & amount_yr2 > 0 & amount_yr3 > 0').count()['id'])
print(f'# who gave year3 recover g/s/g/g: ', new_donations.query('amount_yr0 >  0 & amount_yr1 == 0 & amount_yr2 > 0 & amount_yr3 > 0').count()['id'])
print(f'# who gave year4: ',new_donations.query('amount_yr4 > 0').count()['id'])


# who gave year0:  2024554
# who gave year1:  243436
# who gave year2:  124288
# who gave year2 again:  67489
# who gave year2 recover:  56799
# who gave year3:  68920
# who gave year4:  36668


In [26]:
ay1=new_donations.query('amount_yr1 > 0')

In [28]:
ay1.groupby('cohort_gift_pattern').count()['id']

cohort_gift_pattern
0,1,          162004
0,1,2,         37639
0,1,2,3,       13781
0,1,2,3,4,     13102
0,1,2,4,        2967
0,1,3,          8134
0,1,3,4,        2815
0,1,4,          2994
Name: id, dtype: int64

In [6]:
gp=new_donations.query('cohort_gift_pattern =="none"')

In [7]:
gp.gift_pattern.value_counts()

1,2,3,4,5,    9881
2,3,4,5,      3220
1,3,4,5,      1898
2,4,5,         837
1,2,3,4,         1
Name: gift_pattern, dtype: int64

In [9]:
gp.query('amount_yr1 == 0')

,cohort_gift_pattern,gift_pattern,amount_2013,amount_2014,amount_2015,amount_2016,amount_2017,amount_yr0,amount_yr1,amount_yr2,...,recovered_multi_year,inconsistent_multi_year,recovered_key_multi_year,lapsed_after_first,lapsed_after_first_long,lapsed_after_second,lapsed_multi_year,lapsed_long_multi_year,lapsed_key_multi_year,lapsed_long
167,none,"1,3,4,5,",8400,0,18500,5000,1000,8400,0,18500,...,0,0,0,0,0,0,0,0,0,0
218,none,"1,3,4,5,",25000,0,10000,5000,5000,25000,0,10000,...,0,0,0,0,0,0,0,0,0,0
235,none,"1,3,4,5,",6450,0,30000,9075,46285,6450,0,30000,...,0,0,0,0,0,0,1,0,0,0
264,none,"2,4,5,",0,10000,0,25647,460559,10000,0,25647,...,0,0,0,0,0,0,0,0,0,0
302,none,"1,3,4,5,",33883,0,26200,10000,11034,33883,0,26200,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460122,none,"1,3,4,5,",7500,0,2500,2500,2500,7500,0,2500,...,0,0,0,0,0,0,1,0,0,0
1462419,none,"1,3,4,5,",6500,0,5000,4933,2000,6500,0,5000,...,0,0,0,0,0,0,1,0,0,0
1501043,none,"1,3,4,5,",2000,0,1000,2500,1000,2000,0,1000,...,0,0,0,0,0,0,1,0,0,0
1548071,none,"1,3,4,5,",5000,0,10000,5000,2500,5000,0,10000,...,0,0,0,0,0,0,1,0,0,0


In [21]:
#first_gift = 
#156616 donors
#1650 gave year_1
#644 gave year_2
#3535 gave year_3
#2699 gave year_4

#0,        149275
#0,1,         463
#0,1,2,       390
#0,1,4,       797
#0,2,         254
#0,3,        3535
#0,4,        1902
fg=new_donations.query('first_gift != 0')

In [22]:
fg

,cohort_gift_pattern,gift_pattern,amount_2013,amount_2014,amount_2015,amount_2016,amount_2017,amount_yr0,amount_yr1,amount_yr2,...,recovered_multi_year,inconsistent_multi_year,recovered_key_multi_year,lapsed_after_first,lapsed_after_first_long,lapsed_after_second,lapsed_multi_year,lapsed_long_multi_year,lapsed_key_multi_year,lapsed_long
31,"0,",none,0,0,0,0,0,2098,0,0,...,0,0,0,0,0,0,0,0,0,0
33,"0,",none,0,0,0,0,0,5000,0,0,...,0,0,0,0,0,0,0,0,0,0
34,"0,",none,0,0,0,0,0,5000,0,0,...,0,0,0,0,0,0,0,0,0,0
35,"0,",none,0,0,0,0,0,6500,0,0,...,0,0,0,0,0,0,0,0,0,0
36,"0,",none,0,0,0,0,0,2500,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024454,"0,",none,0,0,0,0,0,3000,0,0,...,0,0,0,0,0,0,0,0,0,0
2024455,"0,",none,0,0,0,0,0,7500,0,0,...,0,0,0,0,0,0,0,0,0,0
2024483,"0,",none,0,0,0,0,0,10000,0,0,...,0,0,0,0,0,0,0,0,0,0
2024484,"0,",none,0,0,0,0,0,5000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
fg.cohort_gift_pattern.value_counts()

0,        149275
0,3,        3535
0,4,        1902
0,1,4,       797
0,1,         463
0,1,2,       390
0,2,         254
Name: cohort_gift_pattern, dtype: int64

In [24]:
sy=new_donations.query('second_year != 0')

In [25]:
sy.cohort_gift_pattern.value_counts()

0,1,      29990
0,3,4       913
0,4,        502
0,1,4,      371
Name: cohort_gift_pattern, dtype: int64

In [26]:
swg=new_donations.query('second_with_gap != 0')

In [27]:
swg.cohort_gift_pattern.value_counts()

0,2,    8338
0,3,     314
Name: cohort_gift_pattern, dtype: int64

In [28]:
my=sy=new_donations.query('multi_year != 0')

In [29]:
my.cohort_gift_pattern.value_counts()

0,1,2,      10335
0,1,3,4       853
none          837
0,3,4         455
0,2,4,        281
0,1,2,4,        1
Name: cohort_gift_pattern, dtype: int64

In [30]:
kmy=new_donations.query('key_multi_year != 0')

In [31]:
kmy.cohort_gift_pattern.value_counts()

none        8939
0,1,2,3,    4597
0,1,3,4      570
Name: cohort_gift_pattern, dtype: int64

In [32]:
rmy=new_donations.query('recovered_multi_year != 0')

In [33]:
rmy.cohort_gift_pattern.value_counts()

0,1,2,4,    628
0,1,3,      199
0,2,3,      181
Name: cohort_gift_pattern, dtype: int64

In [35]:
imy=new_donations.query('inconsistent_multi_year != 0')

UndefinedVariableError: name 'inconsistent_multi_year' is not defined

In [36]:
rkm=new_donations.query('recovered_key_multi_year != 0')

In [37]:
rkm.cohort_gift_pattern.value_counts()

0,1,2,3,    518
Name: cohort_gift_pattern, dtype: int64

In [29]:
cgp=new_donations.groupby('cohort_gift_pattern').sum()
# [('first_gift',
# 'second_year',
# 'second_with_gap',
# 'multi_year',
# 'key_multi_year',
# 'recovered_multi_year',
# 'inconsistent_multi_year ',
# 'recovered_key_multi_year',
# 'lapsed_after_first',
# 'lapsed_after_first_long',
# 'lapsed_after_second',
# 'lapsed_multi_year',
# 'lapsed_long_multi_year',
# 'lapsed_key_multi_year',
# 'lapsed_long')]

In [33]:
cgp.first_gift_year.value_counts()

AttributeError: 'DataFrame' object has no attribute 'first_gift_year'

In [48]:
cgp.to_csv('cgp.csv')